# C interpreter

### 1. First aproximation

Design a agrammar using an upward translation that accepts the following inputs from the C language:

- Assignment statements `(a = 5, a = b = c = 5)`
- Comparison operators `(==, <=, >=, !=)`
- Logical Operators `(&&, ||, !)`
- Arithmetic operators `(+, - , *, / and unary minus)`
- Variables and numerical constants `(a, 5)`



# Grammar design
First to design our grammar we have to now the precedence and the associativity of the operators.

## C operator precedence

The C operator predecence that we have to implement in our grammar is in this precedence order following [C operator Precedence](https://en.cppreference.com/w/c/language/operator_precedence) found in cppreference website:

1. Variables and numerical constants
2. Unary minus and Logical NOT
3. Arithmetic operators    
    - Multiplication, division, and reminder
    - addition and subtraction
4. Comparison operators
    - lt, gt, let, get
    - equals and not equals
4. Logical operators
    - Logical AND 
    - Logical OR
5.  Assignment statements

## Operator Associativity

In the same website we can see the operator associativity in summary:

### Right to Left

- Logical NOT and unary minus
- Simple asisignment

### Left to Right

- Arithmetic operator
- Comparison operators 
- Logical Operators 


## Tokens and non-terminal symbols

The non terminal symbols and tokens that we need to represent our language are:
- ID: to represent the variables `[a-zA-Z][\w_]*`
- CNUM: Numeric constants to represent the variable values or literal values, integer values for now.
- Operators: `[==, <=, >=, !=, &&, ||, !, +, - , *, / and unary minus]`

## Grammar design

- def -> asign `;`
- asign -> ID `=` asign
- asign -> expr
- expr -> exprOR
- exprOR -> exprOR `||` exprAND
- exprOR -> exprAND
- exprAND -> exprAND `&&` exprE
- exprAND -> exprE
- exprE -> exprE `[==, !=]` exprC
- exprE -> exprC
- exprC -> exprC `[<, <=, >, >=]` exprA
- exprC -> exprA
- exprA -> exprA `[+, -] ` add
- exprA -> add
- add -> add `[*, /]` fact
- add -> fact
- fact -> `-` fact
- fact -> num
- fact -> `ID`


## Requirements for implementation

- Install [sly](https://sly.readthedocs.io/en/latest/) library: We will use this library to implement the interpreter.
- Install [pandas](https://pandas.pydata.org/): We will use this library to import/represent data.

In [5]:
!pip install sly
!pip install pandas

You should consider upgrading via the &#39;c:\users\los_g\onedrive\documentos\pl\pl-2020\prácticas\c-interpreter\.env\scripts\python.exe -m pip install --upgrade pip&#39; command.


## Implementing the lexical analyzer (Lexer)

In [117]:
from sly import Lexer

class Scanner(Lexer):
    tokens = {ID, CNUM, EQ, NEQ, OR, AND, LET, GET}
    literals ={'=', '<', '>', '/', '!', '+', '-' , '*'}

    # Ignore whitespace and tabulations

    ignore = ' \t'

    # Regular expressions rules for tokens

    ID = r'[a-zA-Z][\w_]*'
    EQ = r'=='
    NEQ = r'!='
    GET = r'>='
    LET = r'<='
    AND = r'&&'
    OR = r'\|\|'
    CNUM = r'[\d]+'

    # Error handling rule

    def error(self, t):
        print('<-'*10,"Illegal character '{}'".format(t.value[0]), '->'*10)
        self.index += 1
        t.type='Illegal'
        t.value =t.value[0]
        return t


## Testing the lexical analyzer (Lexer)

In [118]:
import pandas as pd 

data = pd.read_csv('../assets/testing/sentences.csv')
data[['sentences']]






,sentences
0,a = 5 + 5
1,5+5
2,5=5
3,=6
4,-5
5,c =-6
6,-9
7,a = b = c = d = r = 5 + 54
8,a = 5 - 54
9,a = -5


In [119]:
lexer = Scanner()
sentences = data['sentences'].values
pass_or_not = []
all_token_pass = True

for index, sentence in enumerate(sentences):
    print('-' * 80,"{} Lexically Testing sentence: '{}'".format(index, sentence),'-' * 80, sep='\n')
    for token in lexer.tokenize(sentence):
        print(" type = '{}', value = '{}'".format(token.type, token.value))
        if all_token_pass and 'Illegal' in token.type:
            all_token_pass = False
    
    pass_or_not.append('Pass') if all_token_pass else pass_or_not.append('FAIL')
    all_token_pass = True

data['Test'] = pass_or_not

--------------------------------------------------------------------------------
0 Lexically Testing sentence: &#39;a = 5 + 5&#39;
--------------------------------------------------------------------------------
 type = &#39;ID&#39;, value = &#39;a&#39;
 type = &#39;=&#39;, value = &#39;=&#39;
 type = &#39;CNUM&#39;, value = &#39;5&#39;
 type = &#39;+&#39;, value = &#39;+&#39;
 type = &#39;CNUM&#39;, value = &#39;5&#39;
--------------------------------------------------------------------------------
1 Lexically Testing sentence: &#39;5+5 &#39;
--------------------------------------------------------------------------------
 type = &#39;CNUM&#39;, value = &#39;5&#39;
 type = &#39;+&#39;, value = &#39;+&#39;
 type = &#39;CNUM&#39;, value = &#39;5&#39;
--------------------------------------------------------------------------------
2 Lexically Testing sentence: &#39;5=5&#39;
--------------------------------------------------------------------------------
 type = &#39;CNUM&#39;, value = &#

In [121]:
data

,sentences,Test
0,a = 5 + 5,Pass
1,5+5,Pass
2,5=5,Pass
3,=6,Pass
4,-5,Pass
5,c =-6,Pass
6,-9,Pass
7,a = b = c = d = r = 5 + 54,Pass
8,a = 5 - 54,Pass
9,a = -5,Pass
